# 🔮 Self-Attention & Transformer Basics

**Phase 4, Step 1: Transformers & Modern Architectures**

---

## 🎯 Цель этого ноутбука

В Phase 3 мы изучили **RNN/LSTM + Attention**:
- ✅ Recurrent архитектуры для последовательностей
- ✅ Attention как механизм взвешивания важности
- ✅ Seq2Seq модели

**Но у RNN есть проблемы:**
- ❌ **Sequential processing**: нельзя параллелить
- ❌ **Vanishing gradients**: сложно учить длинные зависимости
- ❌ **Slow training**: обработка по одному timestep

---

## 🚀 Enter Transformers (2017)

**"Attention is All You Need"** (Vaswani et al., 2017)

**Ключевая идея:** Полностью избавиться от рекуррентности!
- ✅ **Self-Attention**: каждый элемент смотрит на все остальные одновременно
- ✅ **Parallelization**: все вычисления параллельны
- ✅ **Long-range dependencies**: прямые связи между любыми элементами
- ✅ **Scalability**: эффективно на GPU/TPU

**Результат:**
- 🏆 SOTA в NLP: BERT, GPT, T5, GPT-3/4
- 🏆 Computer Vision: ViT (Vision Transformer), DINO
- 🏆 Tabular Data: TabTransformer, FT-Transformer
- 🏆 Time Series: Temporal Fusion Transformer
- 🏆 Multi-modal: CLIP, Flamingo

---

## 📚 Что мы изучим

### 1. Self-Attention Mechanism
- **Query, Key, Value (Q, K, V)**: как работает attention
- **Scaled Dot-Product Attention**: математика
- **Attention Weights**: что модель "смотрит"
- **Implementation**: с нуля в PyTorch

### 2. Multi-Head Attention
- **Multiple attention heads**: параллельные "perspectives"
- **Concatenation & projection**: объединение heads
- **Why it works**: разные heads учат разные паттерны

### 3. Positional Encoding
- **Problem**: Self-Attention permutation-invariant
- **Solution**: добавить информацию о позиции
- **Sinusoidal encoding**: для sequences
- **Learnable embeddings**: для табличных данных

### 4. Transformer Encoder for Tabular Data
- **Dataset**: Titanic (классификация выживших)
- **Architecture**: Feature Embedding → Multi-Head Attention → FFN
- **Training**: Cross-Entropy Loss
- **Evaluation**: сравнение с XGBoost, LSTM
- **Interpretability**: визуализация attention weights

---

## 🔍 Почему Transformers для табличных данных?

**Традиционный подход:**
- Tree-based (XGBoost, LightGBM): хорошо для табличных данных
- MLPs: baseline

**Transformers дают:**
- ✅ **Feature interactions**: автоматическое изучение взаимодействий
- ✅ **Attention weights**: интерпретируемость
- ✅ **Transfer learning**: pre-training на больших датасетах
- ✅ **Mixed data types**: категориальные + числовые

**Когда использовать:**
- 📊 Большие датасеты (>10k samples)
- 📊 Много categorical features
- 📊 Сложные feature interactions
- 📊 Нужна интерпретируемость

---


## 💻 Часть 1: Теория и имплементация Self-Attention

### 1.1 Импорт библиотек

In [ ]:
# Базовые библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, roc_auc_score
)

# Math
import math

# Настройки
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

# Reproducibility
torch.manual_seed(42)
np.random.seed(42)

print("\n✅ Все библиотеки загружены")

### 1.2 Теория: Что такое Self-Attention?

---

## 🧠 Интуиция

**Представьте предложение:** "The animal didn't cross the street because **it** was too tired."

**Вопрос:** На что ссылается "it"?
- Ответ: "The animal" (а не "street")

**Self-Attention делает именно это:**
- Для каждого слова смотрит на **все остальные слова**
- Вычисляет **веса важности** (насколько важно каждое слово)
- Создает **контекстное представление** взвешенной суммой

---

## 📐 Математика: Scaled Dot-Product Attention

**Input:**
- Sequence: $X = [x_1, x_2, ..., x_n]$, где $x_i \in \mathbb{R}^{d}$

**Шаг 1: Создаем Q, K, V (Query, Key, Value)**

$$Q = XW^Q, \quad K = XW^K, \quad V = XW^V$$

Где:
- $W^Q, W^K, W^V \in \mathbb{R}^{d \times d_k}$ - learnable matrices
- $Q, K, V \in \mathbb{R}^{n \times d_k}$

**Интуиция:**
- **Query (Q)**: "Что я ищу?" (запрос от текущего элемента)
- **Key (K)**: "Что я могу предложить?" (описание других элементов)
- **Value (V)**: "Какую информацию я несу?" (actual content)

**Шаг 2: Вычисляем Attention Scores**

$$\text{scores} = \frac{QK^T}{\sqrt{d_k}}$$

- $QK^T$: similarity между queries и keys (dot product)
- $\sqrt{d_k}$: scaling для стабильности градиентов

**Почему scaling?**
- Без scaling: для больших $d_k$, dot products огромные
- Огромные scores → softmax saturation → vanishing gradients
- $\sqrt{d_k}$ нормализует variance

**Шаг 3: Softmax для весов**

$$\text{weights} = \text{softmax}(\text{scores}) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)$$

- Преобразует scores в вероятности: $\sum_i w_i = 1$
- Высокие scores → высокие веса

**Шаг 4: Weighted Sum**

$$\text{Attention}(Q, K, V) = \text{weights} \cdot V = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right) V$$

- Каждый output - взвешенная комбинация всех values
- Веса определяют, сколько каждый элемент "смотрит" на другие

---

## 🎨 Визуальная интуиция

```
Input:     [x1]  [x2]  [x3]  [x4]
              ↓     ↓     ↓     ↓
           [Q1]  [Q2]  [Q3]  [Q4]  ← Queries ("что я ищу?")
           [K1]  [K2]  [K3]  [K4]  ← Keys ("что я предлагаю?")
           [V1]  [V2]  [V3]  [V4]  ← Values (actual info)

Attention for x1:
  Q1 · K1 → score11  ┐
  Q1 · K2 → score12  ├→ softmax → [w11, w12, w13, w14]
  Q1 · K3 → score13  │
  Q1 · K4 → score14  ┘

Output: y1 = w11*V1 + w12*V2 + w13*V3 + w14*V4
```

---

## 🔑 Ключевые свойства

1. **Permutation Invariant (без Positional Encoding):**
   - Attention не зависит от порядка входов
   - $[x_1, x_2, x_3] \equiv [x_3, x_1, x_2]$
   - Нужно добавлять positional encoding!

2. **Parallelizable:**
   - Все attention scores вычисляются одновременно
   - Матричное умножение $QK^T$ - одна операция
   - Нет sequential dependencies (в отличие от RNN)

3. **Long-range Dependencies:**
   - Прямая связь между любыми элементами
   - O(1) path length (vs O(n) в RNN)

4. **Computational Complexity:**
   - $O(n^2 \cdot d)$ для sequence length $n$
   - Bottleneck для очень длинных последовательностей
   - Решение: Sparse Attention, Linformer, etc.

---


### 1.3 Implementation: Scaled Dot-Product Attention

Имплементируем с нуля!

In [ ]:
class ScaledDotProductAttention(nn.Module):
    """
    Scaled Dot-Product Attention
    
    Attention(Q, K, V) = softmax(QK^T / sqrt(d_k)) V
    """
    def __init__(self, dropout=0.1):
        super(ScaledDotProductAttention, self).__init__()
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, Q, K, V, mask=None):
        """
        Args:
            Q: Query matrix (batch_size, n_heads, seq_len, d_k)
            K: Key matrix (batch_size, n_heads, seq_len, d_k)
            V: Value matrix (batch_size, n_heads, seq_len, d_v)
            mask: Mask matrix (optional)
        
        Returns:
            context: Attention output (batch_size, n_heads, seq_len, d_v)
            attention_weights: Attention weights (batch_size, n_heads, seq_len, seq_len)
        """
        # d_k: dimension of keys/queries
        d_k = Q.size(-1)
        
        # Шаг 1: Compute attention scores
        # scores shape: (batch_size, n_heads, seq_len, seq_len)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_k)
        
        # Шаг 2: Apply mask (если есть)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        
        # Шаг 3: Apply softmax
        attention_weights = F.softmax(scores, dim=-1)
        attention_weights = self.dropout(attention_weights)
        
        # Шаг 4: Weighted sum of values
        context = torch.matmul(attention_weights, V)
        
        return context, attention_weights

print("✅ ScaledDotProductAttention реализован!")

### 1.4 Пример: Attention на простых данных

Создадим маленький пример, чтобы понять, как работает attention.

In [ ]:
# Создаем простой пример
batch_size = 1
n_heads = 1  # пока один head
seq_len = 4  # 4 элемента в последовательности
d_k = 8      # размерность keys/queries

# Случайные Q, K, V
torch.manual_seed(42)
Q = torch.randn(batch_size, n_heads, seq_len, d_k)
K = torch.randn(batch_size, n_heads, seq_len, d_k)
V = torch.randn(batch_size, n_heads, seq_len, d_k)

print(f"Q shape: {Q.shape}")
print(f"K shape: {K.shape}")
print(f"V shape: {V.shape}")

# Применяем attention
attention_layer = ScaledDotProductAttention(dropout=0.0)
context, attention_weights = attention_layer(Q, K, V)

print(f"\nContext shape: {context.shape}")
print(f"Attention weights shape: {attention_weights.shape}")

# Визуализируем attention weights
weights = attention_weights[0, 0].detach().numpy()  # (seq_len, seq_len)

plt.figure(figsize=(8, 6))
sns.heatmap(weights, annot=True, fmt='.3f', cmap='YlOrRd', 
            xticklabels=[f'K{i+1}' for i in range(seq_len)],
            yticklabels=[f'Q{i+1}' for i in range(seq_len)],
            cbar_kws={'label': 'Attention Weight'})
plt.title('Attention Weights Matrix', fontsize=16, fontweight='bold')
plt.xlabel('Keys (what to attend to)', fontsize=12)
plt.ylabel('Queries (who is attending)', fontsize=12)
plt.tight_layout()
plt.show()

print("\n📊 Интерпретация:")
print("  - Каждая строка: как Q_i распределяет внимание на все Keys")
print("  - Каждая строка суммируется в 1.0 (softmax property)")
print("  - Высокие веса: Q_i сильно \"смотрит\" на K_j")

# Проверяем, что строки суммируются в 1
row_sums = weights.sum(axis=1)
print(f"\n✅ Проверка softmax: Row sums = {row_sums}")

---

## 🎯 Часть 2: Multi-Head Attention

### 2.1 Теория: Зачем нужны Multiple Heads?

---

## 🤔 Проблема Single-Head Attention

**Single head:**
- Учит только **один паттерн** attention
- Может упустить разные типы взаимодействий

**Пример в NLP:**
- Предложение: "The cat sat on the mat"
- Head 1 может учить **syntactic relationships** (subject-verb)
- Head 2 может учить **semantic relationships** (cat-mat: location)
- Head 3 может учить **long-range dependencies**

**Решение: Multi-Head Attention!**

---

## 📐 Математика Multi-Head Attention

**Идея:** Запускаем $h$ параллельных attention layers с разными проекциями.

**Для каждого head $i$:**

$$\text{head}_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)$$

Где:
- $W_i^Q, W_i^K, W_i^V$ - learnable projection matrices для head $i$
- Обычно: $d_k = d_v = d_{model} / h$ (делим размерность между heads)

**Concatenation:**

$$\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, ..., \text{head}_h) W^O$$

Где:
- $W^O \in \mathbb{R}^{d_{model} \times d_{model}}$ - output projection matrix

**Размерности:**
- Input: $(batch, seq\_len, d_{model})$
- Each head output: $(batch, seq\_len, d_k)$
- Concatenated: $(batch, seq\_len, h \cdot d_k) = (batch, seq\_len, d_{model})$
- Final output: $(batch, seq\_len, d_{model})$

---

## 🎨 Визуализация

```
Input X (d_model)
        ↓
    ┌───┴───┬───────┬───────┐
    ↓       ↓       ↓       ↓
  Head1   Head2   Head3  ... Head_h
  (d_k)   (d_k)   (d_k)     (d_k)
    ↓       ↓       ↓       ↓
   Att1    Att2    Att3   ... Att_h
    └───┬───┴───────┴───────┘
        ↓
    Concat (h * d_k = d_model)
        ↓
    Linear (W^O)
        ↓
    Output (d_model)
```

---

## ✨ Преимущества Multi-Head

1. **Diverse Representations:**
   - Разные heads учат разные паттерны
   - Ensemble effect внутри одного слоя

2. **Richer Feature Space:**
   - $h$ разных проекций → больше способов комбинировать информацию
   - Аналог "multiple filters" в CNN

3. **Interpretability:**
   - Можно визуализировать, что каждый head "смотрит"
   - Разные heads могут специализироваться

4. **Empirical Success:**
   - BERT: 12 heads, GPT-3: 96 heads
   - Критично для SOTA performance

---


### 2.2 Implementation: Multi-Head Attention

In [ ]:
class MultiHeadAttention(nn.Module):
    """
    Multi-Head Attention Layer
    
    Applies h parallel attention heads and concatenates results.
    """
    def __init__(self, d_model, n_heads, dropout=0.1):
        """
        Args:
            d_model: Model dimension (e.g., 512)
            n_heads: Number of attention heads (e.g., 8)
            dropout: Dropout rate
        """
        super(MultiHeadAttention, self).__init__()
        
        assert d_model % n_heads == 0, "d_model must be divisible by n_heads"
        
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = d_model // n_heads  # dimension per head
        
        # Linear projections for Q, K, V
        self.W_Q = nn.Linear(d_model, d_model)
        self.W_K = nn.Linear(d_model, d_model)
        self.W_V = nn.Linear(d_model, d_model)
        
        # Scaled Dot-Product Attention
        self.attention = ScaledDotProductAttention(dropout)
        
        # Output projection
        self.W_O = nn.Linear(d_model, d_model)
        
        self.dropout = nn.Dropout(dropout)
    
    def split_heads(self, x):
        """
        Split last dimension into (n_heads, d_k)
        
        Args:
            x: (batch_size, seq_len, d_model)
        
        Returns:
            (batch_size, n_heads, seq_len, d_k)
        """
        batch_size, seq_len, d_model = x.size()
        return x.view(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)
    
    def combine_heads(self, x):
        """
        Combine heads back
        
        Args:
            x: (batch_size, n_heads, seq_len, d_k)
        
        Returns:
            (batch_size, seq_len, d_model)
        """
        batch_size, n_heads, seq_len, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)
    
    def forward(self, Q, K, V, mask=None):
        """
        Args:
            Q, K, V: (batch_size, seq_len, d_model)
            mask: Optional mask
        
        Returns:
            output: (batch_size, seq_len, d_model)
            attention_weights: (batch_size, n_heads, seq_len, seq_len)
        """
        batch_size = Q.size(0)
        
        # 1. Linear projections
        Q = self.W_Q(Q)  # (batch, seq_len, d_model)
        K = self.W_K(K)
        V = self.W_V(V)
        
        # 2. Split into multiple heads
        Q = self.split_heads(Q)  # (batch, n_heads, seq_len, d_k)
        K = self.split_heads(K)
        V = self.split_heads(V)
        
        # 3. Apply attention on all heads in parallel
        context, attention_weights = self.attention(Q, K, V, mask)
        # context: (batch, n_heads, seq_len, d_k)
        # attention_weights: (batch, n_heads, seq_len, seq_len)
        
        # 4. Concatenate heads
        context = self.combine_heads(context)  # (batch, seq_len, d_model)
        
        # 5. Final linear projection
        output = self.W_O(context)
        output = self.dropout(output)
        
        return output, attention_weights

print("✅ MultiHeadAttention реализован!")

### 2.3 Пример: Multi-Head Attention в действии

In [ ]:
# Параметры
batch_size = 2
seq_len = 6
d_model = 64
n_heads = 4

# Создаем случайный input
torch.manual_seed(42)
x = torch.randn(batch_size, seq_len, d_model)

print(f"Input shape: {x.shape}")
print(f"  batch_size={batch_size}, seq_len={seq_len}, d_model={d_model}")
print(f"  n_heads={n_heads}, d_k per head={d_model // n_heads}")

# Multi-Head Attention
mha = MultiHeadAttention(d_model, n_heads, dropout=0.0)
output, attention_weights = mha(x, x, x)  # Self-Attention: Q=K=V=x

print(f"\nOutput shape: {output.shape}")
print(f"Attention weights shape: {attention_weights.shape}")
print(f"  {n_heads} heads, each with ({seq_len} x {seq_len}) attention matrix")

# Визуализация attention weights всех heads
fig, axes = plt.subplots(1, n_heads, figsize=(16, 4))

for i in range(n_heads):
    weights = attention_weights[0, i].detach().numpy()  # первый sample, i-й head
    
    sns.heatmap(weights, annot=True, fmt='.2f', cmap='YlOrRd', 
                ax=axes[i], cbar=False, square=True,
                xticklabels=range(1, seq_len+1),
                yticklabels=range(1, seq_len+1))
    axes[i].set_title(f'Head {i+1}', fontsize=14, fontweight='bold')
    axes[i].set_xlabel('Keys')
    if i == 0:
        axes[i].set_ylabel('Queries')

plt.suptitle('Multi-Head Attention Weights (4 heads)', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\n📊 Наблюдения:")
print("  - Каждый head учит свой паттерн attention")
print("  - Heads могут focus на разных positions")
print("  - Некоторые heads более \"diagonal\" (local), другие более \"distributed\" (global)")

---

## 📍 Часть 3: Positional Encoding

### 3.1 Теория: Проблема Permutation Invariance

---

## ⚠️ Проблема

**Self-Attention permutation invariant:**

$$\text{Attention}([x_1, x_2, x_3]) = \text{Attention}([x_3, x_1, x_2])$$

**Почему это проблема?**
- Порядок элементов важен! (особенно для sequences)
- "cat sat on mat" ≠ "mat on sat cat"
- Модель не знает position каждого элемента

**Решение: Positional Encoding!**

---

## 📐 Sinusoidal Positional Encoding (для sequences)

**Идея:** Добавить к каждому элементу уникальный positional vector.

$$\text{PE}(pos, 2i) = \sin\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$

$$\text{PE}(pos, 2i+1) = \cos\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$

Где:
- $pos$: позиция в последовательности (0, 1, 2, ...)
- $i$: dimension index (0, 1, ..., $d_{model}/2$)
- Even dimensions: sine
- Odd dimensions: cosine

**Свойства:**
1. **Unique**: каждая позиция → уникальный вектор
2. **Bounded**: значения в $[-1, 1]$
3. **Relative positions**: $\text{PE}(pos+k)$ - linear function of $\text{PE}(pos)$
4. **Extrapolation**: работает для позиций, не виденных при обучении

**Почему sin/cos?**
- Разные частоты для разных dimensions
- Low dimensions: медленные колебания (long-range)
- High dimensions: быстрые колебания (short-range)
- Модель сама выбирает, какие частоты использовать

---

## 🗂️ Learnable Positional Embeddings (для табличных данных)

**Для табличных данных:**
- Нет "естественного" порядка features
- Можем использовать **learnable embeddings**:

$$\text{PE}_i \in \mathbb{R}^{d_{model}}$$

- Просто learnable parameters для каждой feature position
- Более гибко, адаптируется к данным

**Final Input:**

$$X_{final} = X_{embedded} + \text{PE}$$

---


### 3.2 Implementation: Positional Encoding

In [ ]:
class PositionalEncoding(nn.Module):
    """
    Sinusoidal Positional Encoding for Sequences
    
    PE(pos, 2i) = sin(pos / 10000^(2i/d_model))
    PE(pos, 2i+1) = cos(pos / 10000^(2i/d_model))
    """
    def __init__(self, d_model, max_len=5000, dropout=0.1):
        """
        Args:
            d_model: Model dimension
            max_len: Maximum sequence length
            dropout: Dropout rate
        """
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(dropout)
        
        # Создаем positional encoding matrix
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        
        # Compute div_term: 10000^(2i/d_model)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * 
                             (-math.log(10000.0) / d_model))
        
        # Apply sin to even indices
        pe[:, 0::2] = torch.sin(position * div_term)
        
        # Apply cos to odd indices
        pe[:, 1::2] = torch.cos(position * div_term)
        
        # Add batch dimension
        pe = pe.unsqueeze(0)  # (1, max_len, d_model)
        
        # Register as buffer (not a parameter, but part of module state)
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        """
        Args:
            x: (batch_size, seq_len, d_model)
        
        Returns:
            x + positional_encoding: (batch_size, seq_len, d_model)
        """
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)


class LearnablePositionalEmbedding(nn.Module):
    """
    Learnable Positional Embeddings for Tabular Data
    """
    def __init__(self, num_features, d_model, dropout=0.1):
        """
        Args:
            num_features: Number of features (sequence length)
            d_model: Model dimension
            dropout: Dropout rate
        """
        super(LearnablePositionalEmbedding, self).__init__()
        self.pos_embedding = nn.Parameter(torch.randn(1, num_features, d_model))
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        """
        Args:
            x: (batch_size, num_features, d_model)
        
        Returns:
            x + positional_embedding: (batch_size, num_features, d_model)
        """
        x = x + self.pos_embedding
        return self.dropout(x)

print("✅ Positional Encoding реализован (Sinusoidal и Learnable)!")

### 3.3 Визуализация Positional Encoding

In [ ]:
# Создаем positional encoding
d_model = 64
max_len = 100

pe_layer = PositionalEncoding(d_model, max_len, dropout=0.0)
pe_matrix = pe_layer.pe[0].numpy()  # (max_len, d_model)

print(f"Positional Encoding shape: {pe_matrix.shape}")

# Визуализация 1: Heatmap
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Heatmap всех позиций и dimensions
im = axes[0].imshow(pe_matrix.T, cmap='RdBu', aspect='auto', interpolation='nearest')
axes[0].set_title('Positional Encoding Matrix', fontsize=16, fontweight='bold')
axes[0].set_xlabel('Position', fontsize=12)
axes[0].set_ylabel('Dimension', fontsize=12)
plt.colorbar(im, ax=axes[0], label='Value')

# Несколько примеров позиций
positions_to_plot = [0, 10, 20, 40, 80]
for pos in positions_to_plot:
    axes[1].plot(pe_matrix[pos], label=f'Position {pos}', alpha=0.7)

axes[1].set_title('Positional Encodings for Different Positions', 
                  fontsize=16, fontweight='bold')
axes[1].set_xlabel('Dimension', fontsize=12)
axes[1].set_ylabel('Value', fontsize=12)
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Наблюдения:")
print("  - Каждая позиция имеет уникальный паттерн")
print("  - Low dimensions: медленные колебания (видны полосы на heatmap)")
print("  - High dimensions: быстрые колебания (более мелкая структура)")
print("  - Модель может использовать эти паттерны для определения relative positions")

---

## 🏗️ Часть 4: Transformer Encoder

### 4.1 Теория: Полная архитектура Encoder

---

## 📐 Transformer Encoder Block

**Структура одного Encoder Block:**

```
Input
  ↓
┌─────────────────────┐
│ Multi-Head Attention│
└──────────┬──────────┘
           ↓
      Add & Norm (Residual + LayerNorm)
           ↓
┌─────────────────────┐
│  Feed Forward (FFN) │
│   (2-layer MLP)     │
└──────────┬──────────┘
           ↓
      Add & Norm (Residual + LayerNorm)
           ↓
        Output
```

**Компоненты:**

1. **Multi-Head Self-Attention:**
   - Q = K = V = Input (self-attention)
   - Feature interactions

2. **Add & Norm (Residual Connection + Layer Normalization):**
   $$\text{LayerNorm}(x + \text{Sublayer}(x))$$
   - Residual: помогает gradient flow
   - LayerNorm: стабилизирует обучение

3. **Feed Forward Network (FFN):**
   $$\text{FFN}(x) = \text{ReLU}(xW_1 + b_1)W_2 + b_2$$
   - 2-layer MLP с ReLU
   - Применяется independently к каждой позиции
   - Обычно: hidden dim = 4 × d_model

4. **Another Add & Norm**

**Stacking Multiple Blocks:**
- Original Transformer: 6 encoder blocks
- BERT-base: 12 blocks
- GPT-3: 96 blocks

---

## 🎯 Для табличных данных

**Архитектура:**

```
Tabular Features [x1, x2, ..., xn]
        ↓
Feature Embedding (Linear projection)
        ↓
Positional Encoding (learnable)
        ↓
Transformer Encoder Blocks (N layers)
        ↓
Global Pooling (mean/max/CLS token)
        ↓
Classification Head (Linear)
        ↓
Output (class probabilities)
```

**Key Differences from NLP:**
- **No word embeddings**: Linear projection вместо embedding lookup
- **Learnable PE**: вместо sinusoidal (нет естественного порядка)
- **Global pooling**: mean/max вместо CLS token (можно и CLS)

---


### 4.2 Implementation: Feed Forward Network

In [ ]:
class FeedForward(nn.Module):
    """
    Position-wise Feed Forward Network
    
    FFN(x) = ReLU(xW1 + b1)W2 + b2
    """
    def __init__(self, d_model, d_ff, dropout=0.1):
        """
        Args:
            d_model: Model dimension
            d_ff: Hidden dimension (usually 4 * d_model)
            dropout: Dropout rate
        """
        super(FeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        """
        Args:
            x: (batch_size, seq_len, d_model)
        
        Returns:
            (batch_size, seq_len, d_model)
        """
        return self.linear2(self.dropout(F.relu(self.linear1(x))))

print("✅ FeedForward реализован!")

### 4.3 Implementation: Transformer Encoder Block

In [ ]:
class TransformerEncoderBlock(nn.Module):
    """
    Single Transformer Encoder Block
    
    Components:
    1. Multi-Head Self-Attention
    2. Add & Norm
    3. Feed Forward Network
    4. Add & Norm
    """
    def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
        """
        Args:
            d_model: Model dimension
            n_heads: Number of attention heads
            d_ff: Feed-forward hidden dimension
            dropout: Dropout rate
        """
        super(TransformerEncoderBlock, self).__init__()
        
        # Multi-Head Attention
        self.attention = MultiHeadAttention(d_model, n_heads, dropout)
        
        # Feed Forward
        self.feed_forward = FeedForward(d_model, d_ff, dropout)
        
        # Layer Normalization
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, mask=None):
        """
        Args:
            x: (batch_size, seq_len, d_model)
            mask: Optional mask
        
        Returns:
            (batch_size, seq_len, d_model)
        """
        # 1. Multi-Head Attention + Add & Norm
        attn_output, attention_weights = self.attention(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        
        # 2. Feed Forward + Add & Norm
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        
        return x, attention_weights

print("✅ TransformerEncoderBlock реализован!")

### 4.4 Implementation: Full Transformer Encoder для табличных данных

In [ ]:
class TabularTransformerEncoder(nn.Module):
    """
    Transformer Encoder для табличных данных
    
    Architecture:
    Input → Feature Embedding → Positional Encoding → 
    Transformer Blocks → Global Pooling → Classification Head
    """
    def __init__(self, num_features, d_model, n_heads, n_layers, d_ff, 
                 num_classes, dropout=0.1, pooling='mean'):
        """
        Args:
            num_features: Number of input features
            d_model: Model dimension
            n_heads: Number of attention heads
            n_layers: Number of Transformer blocks
            d_ff: Feed-forward hidden dimension
            num_classes: Number of output classes
            dropout: Dropout rate
            pooling: Pooling method ('mean', 'max', 'cls')
        """
        super(TabularTransformerEncoder, self).__init__()
        
        self.num_features = num_features
        self.d_model = d_model
        self.pooling = pooling
        
        # Feature embedding (project each feature to d_model)
        self.feature_embedding = nn.Linear(1, d_model)  # each feature independently
        
        # Positional encoding (learnable)
        self.pos_encoding = LearnablePositionalEmbedding(num_features, d_model, dropout)
        
        # Stack of Transformer Encoder blocks
        self.encoder_blocks = nn.ModuleList([
            TransformerEncoderBlock(d_model, n_heads, d_ff, dropout)
            for _ in range(n_layers)
        ])
        
        # CLS token (if using cls pooling)
        if pooling == 'cls':
            self.cls_token = nn.Parameter(torch.randn(1, 1, d_model))
        
        # Classification head
        self.classifier = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model // 2, num_classes)
        )
    
    def forward(self, x, return_attention=False):
        """
        Args:
            x: (batch_size, num_features)
            return_attention: Whether to return attention weights
        
        Returns:
            logits: (batch_size, num_classes)
            attention_weights: Optional, list of attention weights from each layer
        """
        batch_size = x.size(0)
        
        # 1. Feature embedding: (batch, num_features) → (batch, num_features, d_model)
        x = x.unsqueeze(-1)  # (batch, num_features, 1)
        x = self.feature_embedding(x)  # (batch, num_features, d_model)
        
        # 2. Add CLS token if using cls pooling
        if self.pooling == 'cls':
            cls_tokens = self.cls_token.expand(batch_size, -1, -1)  # (batch, 1, d_model)
            x = torch.cat([cls_tokens, x], dim=1)  # (batch, num_features+1, d_model)
        
        # 3. Add positional encoding
        x = self.pos_encoding(x)
        
        # 4. Pass through Transformer blocks
        attention_weights_list = []
        for encoder_block in self.encoder_blocks:
            x, attn_weights = encoder_block(x)
            if return_attention:
                attention_weights_list.append(attn_weights)
        
        # 5. Global pooling
        if self.pooling == 'mean':
            x = x.mean(dim=1)  # (batch, d_model)
        elif self.pooling == 'max':
            x = x.max(dim=1)[0]  # (batch, d_model)
        elif self.pooling == 'cls':
            x = x[:, 0, :]  # take CLS token
        
        # 6. Classification
        logits = self.classifier(x)
        
        if return_attention:
            return logits, attention_weights_list
        return logits

print("✅ TabularTransformerEncoder реализован!")
print("\nАрхитектура готова:")
print("  1. Feature Embedding (Linear projection)")
print("  2. Learnable Positional Encoding")
print("  3. Stack of Transformer Encoder Blocks")
print("  4. Global Pooling (mean/max/cls)")
print("  5. Classification Head")

---

## 📊 Часть 5: Titanic Dataset

### 5.1 Загрузка и подготовка данных

**Задача:** Предсказать выживание пассажиров Титаника  
**Тип:** Бинарная классификация (Survived: 0 или 1)  
**Features:** Pclass, Sex, Age, SibSp, Parch, Fare, Embarked


In [ ]:
# Загрузка Titanic dataset
# Используем встроенный датасет из seaborn

df = sns.load_dataset('titanic')

print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
print(df.head())

print(f"\nColumns: {df.columns.tolist()}")
print(f"\nMissing values:")
print(df.isnull().sum())

In [ ]:
# Feature Engineering

# Выбираем нужные features
features_to_use = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'alone']
target = 'survived'

# Создаем копию
data = df[features_to_use + [target]].copy()

# Handle missing values
data['age'].fillna(data['age'].median(), inplace=True)
data['fare'].fillna(data['fare'].median(), inplace=True)
data['embarked'].fillna(data['embarked'].mode()[0], inplace=True)

# Encode categorical variables
# Sex: male=1, female=0
data['sex'] = (data['sex'] == 'male').astype(int)

# Embarked: one-hot encoding
data = pd.get_dummies(data, columns=['embarked'], prefix='embarked', drop_first=True)

# alone: boolean to int
data['alone'] = data['alone'].astype(int)

print(f"After preprocessing: {data.shape}")
print(f"\nFeatures: {[col for col in data.columns if col != 'survived']}")
print(f"\nMissing values: {data.isnull().sum().sum()}")

# Check class balance
print(f"\nClass distribution:")
print(data['survived'].value_counts())
print(f"\nSurvival rate: {data['survived'].mean():.2%}")

In [ ]:
# Visualize data
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# Survival by class
pd.crosstab(data['pclass'], data['survived']).plot(kind='bar', ax=axes[0, 0])
axes[0, 0].set_title('Survival by Class', fontweight='bold')
axes[0, 0].set_xlabel('Pclass')
axes[0, 0].set_ylabel('Count')
axes[0, 0].legend(['Died', 'Survived'])

# Survival by sex
pd.crosstab(data['sex'], data['survived']).plot(kind='bar', ax=axes[0, 1])
axes[0, 1].set_title('Survival by Sex', fontweight='bold')
axes[0, 1].set_xlabel('Sex (0=female, 1=male)')
axes[0, 1].set_xticklabels(['Female', 'Male'], rotation=0)
axes[0, 1].legend(['Died', 'Survived'])

# Age distribution
data[data['survived'] == 0]['age'].hist(bins=30, alpha=0.5, label='Died', ax=axes[0, 2])
data[data['survived'] == 1]['age'].hist(bins=30, alpha=0.5, label='Survived', ax=axes[0, 2])
axes[0, 2].set_title('Age Distribution', fontweight='bold')
axes[0, 2].set_xlabel('Age')
axes[0, 2].legend()

# Fare distribution
data[data['survived'] == 0]['fare'].hist(bins=30, alpha=0.5, label='Died', ax=axes[1, 0])
data[data['survived'] == 1]['fare'].hist(bins=30, alpha=0.5, label='Survived', ax=axes[1, 0])
axes[1, 0].set_title('Fare Distribution', fontweight='bold')
axes[1, 0].set_xlabel('Fare')
axes[1, 0].legend()

# SibSp distribution
pd.crosstab(data['sibsp'], data['survived']).plot(kind='bar', ax=axes[1, 1])
axes[1, 1].set_title('Survival by SibSp', fontweight='bold')
axes[1, 1].set_xlabel('Siblings/Spouses')
axes[1, 1].legend(['Died', 'Survived'])

# Correlation heatmap
corr = data.corr()
sns.heatmap(corr[['survived']].sort_values('survived', ascending=False), 
            annot=True, fmt='.2f', cmap='RdYlGn', ax=axes[1, 2], cbar=False)
axes[1, 2].set_title('Correlation with Survival', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 Key insights:")
print("  - Women had much higher survival rate")
print("  - First class passengers survived more")
print("  - Children had better survival chances")
print("  - Higher fare → higher survival (proxy for class)")

In [ ]:
# Train/Test Split

# Separate features and target
X = data.drop('survived', axis=1).values
y = data['survived'].values

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTrain set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("\n✅ Features standardized")

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test)

# Create DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"\n✅ DataLoaders created (batch_size={batch_size})")
print(f"Number of features: {X_train.shape[1]}")

---

## 🏋️ Часть 6: Обучение Transformer

### 6.1 Инициализация модели

In [ ]:
# Hyperparameters
num_features = X_train.shape[1]
d_model = 64
n_heads = 4
n_layers = 2
d_ff = 256  # 4 * d_model
num_classes = 2
dropout = 0.1
pooling = 'mean'  # 'mean', 'max', or 'cls'

print("Model Hyperparameters:")
print(f"  Input features: {num_features}")
print(f"  d_model: {d_model}")
print(f"  n_heads: {n_heads}")
print(f"  n_layers: {n_layers}")
print(f"  d_ff: {d_ff}")
print(f"  dropout: {dropout}")
print(f"  pooling: {pooling}")
print(f"  num_classes: {num_classes}")

# Initialize model
model = TabularTransformerEncoder(
    num_features=num_features,
    d_model=d_model,
    n_heads=n_heads,
    n_layers=n_layers,
    d_ff=d_ff,
    num_classes=num_classes,
    dropout=dropout,
    pooling=pooling
).to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\nModel size:")
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                   factor=0.5, patience=5)

print("\n✅ Model initialized!")

### 6.2 Training Loop

In [ ]:
# Training function
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for X_batch, y_batch in loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Metrics
        total_loss += loss.item() * X_batch.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)
    
    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy


# Evaluation function
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            
            total_loss += loss.item() * X_batch.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == y_batch).sum().item()
            total += y_batch.size(0)
    
    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

print("✅ Training functions defined")

In [ ]:
# Training loop
num_epochs = 50
history = {
    'train_loss': [],
    'train_acc': [],
    'test_loss': [],
    'test_acc': []
}

print(f"Training for {num_epochs} epochs...\n")
best_test_acc = 0

for epoch in range(num_epochs):
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # Evaluate
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    
    # Scheduler step
    scheduler.step(test_loss)
    
    # Save history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['test_loss'].append(test_loss)
    history['test_acc'].append(test_acc)
    
    # Track best
    if test_acc > best_test_acc:
        best_test_acc = test_acc
    
    # Print progress
    if (epoch + 1) % 10 == 0 or epoch == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}]")
        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"  Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")
        print(f"  LR: {optimizer.param_groups[0]['lr']:.6f}")
        print()

print(f"\n✅ Training completed!")
print(f"Best Test Accuracy: {best_test_acc:.4f}")

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
axes[0].plot(history['train_loss'], label='Train Loss', linewidth=2)
axes[0].plot(history['test_loss'], label='Test Loss', linewidth=2)
axes[0].set_title('Loss over Epochs', fontsize=16, fontweight='bold')
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].legend()
axes[0].grid(alpha=0.3)

# Accuracy
axes[1].plot(history['train_acc'], label='Train Accuracy', linewidth=2)
axes[1].plot(history['test_acc'], label='Test Accuracy', linewidth=2)
axes[1].set_title('Accuracy over Epochs', fontsize=16, fontweight='bold')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy', fontsize=12)
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Final Train Accuracy: {history['train_acc'][-1]:.4f}")
print(f"Final Test Accuracy: {history['test_acc'][-1]:.4f}")
print(f"Best Test Accuracy: {best_test_acc:.4f}")

### 6.3 Detailed Evaluation

In [ ]:
# Get predictions
model.eval()
y_pred = []
y_true = []
y_probs = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        probs = F.softmax(outputs, dim=1)
        _, predicted = torch.max(outputs, 1)
        
        y_pred.extend(predicted.cpu().numpy())
        y_true.extend(y_batch.numpy())
        y_probs.extend(probs[:, 1].cpu().numpy())  # probability of class 1

y_pred = np.array(y_pred)
y_true = np.array(y_true)
y_probs = np.array(y_probs)

# Metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
auc = roc_auc_score(y_true, y_probs)

print("Test Set Performance:")
print(f"  Accuracy:  {accuracy:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall:    {recall:.4f}")
print(f"  F1 Score:  {f1:.4f}")
print(f"  ROC AUC:   {auc:.4f}")

# Classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['Died', 'Survived']))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Died', 'Survived'],
            yticklabels=['Died', 'Survived'],
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix', fontsize=16, fontweight='bold')
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('True', fontsize=12)
plt.tight_layout()
plt.show()

---

## 🔍 Часть 7: Attention Visualization

### 7.1 Визуализация Attention Weights

**Что мы ищем:**
- Какие features модель считает важными?
- Есть ли паттерны в attention между features?
- Как разные heads специализируются?

In [ ]:
# Выбираем несколько примеров для визуализации
sample_indices = [0, 5, 10]  # индексы в test set

# Feature names
feature_names = [col for col in data.columns if col != 'survived']
print(f"Features ({len(feature_names)}): {feature_names}")

# Get attention weights для выбранных примеров
model.eval()
with torch.no_grad():
    for idx in sample_indices:
        x_sample = X_test_tensor[idx:idx+1].to(device)
        y_sample = y_test_tensor[idx].item()
        
        # Forward pass with attention
        logits, attention_list = model(x_sample, return_attention=True)
        pred_prob = F.softmax(logits, dim=1)[0, 1].item()
        pred_class = torch.argmax(logits, dim=1).item()
        
        print(f"\n{'='*60}")
        print(f"Sample {idx}: True={y_sample}, Pred={pred_class}, Prob={pred_prob:.3f}")
        print(f"{'='*60}")
        
        # Визуализируем attention weights для всех layers
        n_layers_viz = len(attention_list)
        n_heads_viz = attention_list[0].size(1)
        
        fig, axes = plt.subplots(n_layers_viz, n_heads_viz, 
                                figsize=(n_heads_viz * 4, n_layers_viz * 4))
        
        if n_layers_viz == 1:
            axes = axes.reshape(1, -1)
        
        for layer_idx in range(n_layers_viz):
            attn_weights = attention_list[layer_idx][0].cpu().numpy()  # (n_heads, seq_len, seq_len)
            
            for head_idx in range(n_heads_viz):
                ax = axes[layer_idx, head_idx]
                
                # Attention matrix для этого head
                attn_matrix = attn_weights[head_idx]  # (seq_len, seq_len)
                
                sns.heatmap(attn_matrix, annot=False, fmt='.2f', cmap='YlOrRd',
                           xticklabels=feature_names, yticklabels=feature_names,
                           ax=ax, cbar=True, square=True)
                ax.set_title(f'Layer {layer_idx+1}, Head {head_idx+1}', 
                           fontsize=12, fontweight='bold')
                ax.set_xlabel('Keys (attending to)')
                if head_idx == 0:
                    ax.set_ylabel('Queries (attending from)')
        
        plt.suptitle(f'Sample {idx}: Attention Weights Across Layers and Heads\n'
                    f'True: {y_sample}, Pred: {pred_class}, Prob: {pred_prob:.3f}',
                    fontsize=16, fontweight='bold', y=1.01)
        plt.tight_layout()
        plt.show()

In [ ]:
# Aggregate attention weights across all test samples
# Для понимания, какие features модель считает важными в среднем

model.eval()
all_attention_weights = []

with torch.no_grad():
    for X_batch, _ in test_loader:
        X_batch = X_batch.to(device)
        _, attention_list = model(X_batch, return_attention=True)
        
        # Берем только последний layer
        last_layer_attn = attention_list[-1]  # (batch, n_heads, seq_len, seq_len)
        
        # Усредняем по heads
        avg_attn = last_layer_attn.mean(dim=1)  # (batch, seq_len, seq_len)
        
        all_attention_weights.append(avg_attn.cpu())

# Concatenate и усредняем по всем samples
all_attention_weights = torch.cat(all_attention_weights, dim=0)
mean_attention = all_attention_weights.mean(dim=0).numpy()  # (seq_len, seq_len)

print(f"Mean attention shape: {mean_attention.shape}")

# Визуализируем среднюю attention matrix
plt.figure(figsize=(10, 8))
sns.heatmap(mean_attention, annot=True, fmt='.2f', cmap='YlOrRd',
           xticklabels=feature_names, yticklabels=feature_names,
           cbar_kws={'label': 'Attention Weight'})
plt.title('Mean Attention Weights (Last Layer, Averaged over Test Set)', 
         fontsize=16, fontweight='bold')
plt.xlabel('Keys (attending to)', fontsize=12)
plt.ylabel('Queries (attending from)', fontsize=12)
plt.tight_layout()
plt.show()

# Вычисляем важность features (сколько на них обращают внимание)
feature_importance = mean_attention.sum(axis=0)  # сумма по columns (куда смотрят)

# Нормализуем
feature_importance = feature_importance / feature_importance.sum()

# Сортируем
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance
}).sort_values('Importance', ascending=False)

print("\nFeature Importance (based on attention):")
print(importance_df)

# Визуализируем
plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'], color='steelblue')
plt.xlabel('Attention-based Importance', fontsize=12)
plt.title('Feature Importance from Attention Weights', fontsize=16, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

---

## 📊 Часть 8: Сравнение с Baseline моделями

### 8.1 Simple MLP Baseline

In [ ]:
# Simple MLP для сравнения
class SimpleMLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_classes, dropout=0.3):
        super(SimpleMLP, self).__init__()
        layers = []
        
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            prev_size = hidden_size
        
        layers.append(nn.Linear(prev_size, num_classes))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

# Train MLP
mlp = SimpleMLP(input_size=num_features, hidden_sizes=[64, 32], 
                num_classes=2, dropout=0.3).to(device)

mlp_optimizer = optim.Adam(mlp.parameters(), lr=0.001, weight_decay=1e-5)
mlp_criterion = nn.CrossEntropyLoss()

print(f"Training MLP for {num_epochs} epochs...")
mlp_best_acc = 0

for epoch in range(num_epochs):
    mlp.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        mlp_optimizer.zero_grad()
        outputs = mlp(X_batch)
        loss = mlp_criterion(outputs, y_batch)
        loss.backward()
        mlp_optimizer.step()
    
    # Evaluate
    if (epoch + 1) % 10 == 0 or epoch == 0:
        _, test_acc = evaluate(mlp, test_loader, mlp_criterion, device)
        if test_acc > mlp_best_acc:
            mlp_best_acc = test_acc
        print(f"Epoch [{epoch+1}/{num_epochs}] MLP Test Acc: {test_acc:.4f}")

print(f"\nMLP Best Test Accuracy: {mlp_best_acc:.4f}")

### 8.2 XGBoost Baseline

In [ ]:
# XGBoost baseline
try:
    import xgboost as xgb
    
    print("Training XGBoost...")
    xgb_model = xgb.XGBClassifier(
        n_estimators=100,
        max_depth=5,
        learning_rate=0.1,
        random_state=42
    )
    
    xgb_model.fit(X_train, y_train)
    xgb_pred = xgb_model.predict(X_test)
    xgb_acc = accuracy_score(y_test, xgb_pred)
    xgb_f1 = f1_score(y_test, xgb_pred)
    
    print(f"XGBoost Test Accuracy: {xgb_acc:.4f}")
    print(f"XGBoost Test F1 Score: {xgb_f1:.4f}")
    
    # Feature importance
    xgb_importance = xgb_model.feature_importances_
    xgb_importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': xgb_importance
    }).sort_values('Importance', ascending=False)
    
    print("\nXGBoost Feature Importance:")
    print(xgb_importance_df)
    
except ImportError:
    print("⚠️ XGBoost not installed. Skipping XGBoost baseline.")
    print("Install with: pip install xgboost")
    xgb_acc = None
    xgb_f1 = None

### 8.3 Model Comparison

In [ ]:
# Compare all models
comparison_data = {
    'Model': ['Transformer', 'MLP', 'XGBoost'],
    'Accuracy': [accuracy, mlp_best_acc, xgb_acc if xgb_acc else 0],
    'F1 Score': [f1, 0, xgb_f1 if xgb_f1 else 0],  # не считали F1 для MLP
    'Parameters': [trainable_params, 
                   sum(p.numel() for p in mlp.parameters()),
                   0]  # XGBoost не имеет градиентных параметров
}

comparison_df = pd.DataFrame(comparison_data)
print("\n" + "="*60)
print("MODEL COMPARISON")
print("="*60)
print(comparison_df.to_string(index=False))
print("="*60)

# Визуализация
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy comparison
axes[0].bar(comparison_df['Model'], comparison_df['Accuracy'], 
           color=['steelblue', 'orange', 'green'])
axes[0].set_title('Model Accuracy Comparison', fontsize=16, fontweight='bold')
axes[0].set_ylabel('Accuracy', fontsize=12)
axes[0].set_ylim([0.7, 0.9])
axes[0].grid(alpha=0.3, axis='y')
for i, v in enumerate(comparison_df['Accuracy']):
    axes[0].text(i, v + 0.01, f"{v:.3f}", ha='center', fontweight='bold')

# Parameters comparison
valid_params = [(m, p) for m, p in zip(comparison_df['Model'], comparison_df['Parameters']) if p > 0]
if valid_params:
    models, params = zip(*valid_params)
    axes[1].bar(models, params, color=['steelblue', 'orange'])
    axes[1].set_title('Model Size (Parameters)', fontsize=16, fontweight='bold')
    axes[1].set_ylabel('# Parameters', fontsize=12)
    axes[1].grid(alpha=0.3, axis='y')
    for i, (m, p) in enumerate(valid_params):
        axes[1].text(i, p + 500, f"{p:,}", ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 Выводы:")
print("  - Transformer показывает competitive результаты с baselines")
print("  - XGBoost по-прежнему силен на табличных данных (особенно на маленьких)")
print("  - Transformer дает бонус: attention weights для interpretability")
print("  - На больших датасетах Transformer может превзойти tree-based методы")

---

## 🎓 Итоги и Выводы

### Что мы изучили

#### 1. Self-Attention Mechanism
- ✅ **Query, Key, Value**: как работает attention
- ✅ **Scaled Dot-Product**: $\text{Attention}(Q,K,V) = \text{softmax}(QK^T/\sqrt{d_k})V$
- ✅ **Parallelization**: все вычисления параллельны (vs RNN sequential)
- ✅ **Long-range dependencies**: прямые связи O(1) path length

#### 2. Multi-Head Attention
- ✅ **Multiple perspectives**: разные heads учат разные паттерны
- ✅ **Ensemble effect**: внутри одного слоя
- ✅ **Richer representations**: $h$ параллельных attention mechanisms

#### 3. Positional Encoding
- ✅ **Sinusoidal**: для sequences (NLP, time series)
- ✅ **Learnable**: для табличных данных (нет естественного порядка)
- ✅ **Решает permutation invariance**: модель знает позиции

#### 4. Transformer Encoder
- ✅ **Architecture**: Multi-Head Attention → Add&Norm → FFN → Add&Norm
- ✅ **Residual connections**: помогают gradient flow
- ✅ **Layer Normalization**: стабилизирует обучение
- ✅ **Feed-Forward Network**: позиционно-независимые трансформации

#### 5. Применение к Табличным Данным
- ✅ **Feature embedding**: Linear projection для каждой feature
- ✅ **Feature interactions**: автоматически через attention
- ✅ **Interpretability**: attention weights показывают важность
- ✅ **Competitive performance**: сравнимо с XGBoost на Titanic

---

### Ключевые преимущества Transformers

| Aspect | RNN/LSTM | Transformer |
|--------|----------|-------------|
| **Processing** | Sequential | Parallel |
| **Long-range deps** | O(n) path | O(1) path |
| **Training speed** | Slow | Fast (GPU) |
| **Interpretability** | Hard | Attention weights |
| **Scalability** | Limited | Excellent |
| **SOTA results** | 2014-2017 | 2017+ |

---

### Когда использовать Transformers для табличных данных?

**✅ Хорошо работают когда:**
- Большие датасеты (>10k samples)
- Много categorical features
- Сложные feature interactions
- Нужна interpretability
- Доступен pre-training на похожих данных

**❌ Могут уступать tree-based методам когда:**
- Маленькие датасеты (<1k samples)
- Простые feature interactions
- Только numerical features
- Нужна максимальная производительность без тюнинга

---

### Что дальше?

**Phase 4, Step 2: TabTransformer**
- Специальные embeddings для категориальных features
- Contextual embeddings через Transformer
- Лучшая производительность на табличных данных

**Phase 4, Step 3: Advanced Architectures**
- FT-Transformer (Feature Tokenizer)
- SAINT (Self-Attention and Intersample Attention)
- TabNet (attention-based табличная архитектура)
- Temporal Fusion Transformer (для time series)

**Phase 5: Transfer Learning & Pre-training**
- Pre-training на больших датасетах
- Fine-tuning на целевой задаче
- Self-supervised learning

---

### Дополнительные ресурсы

**Papers:**
- "Attention is All You Need" (Vaswani et al., 2017) - оригинальная статья
- "TabTransformer" (Huang et al., 2020) - Transformer для табличных данных
- "Revisiting Deep Learning Models for Tabular Data" (Gorishniy et al., 2021)

**Tutorials:**
- The Illustrated Transformer (Jay Alammar)
- Annotated Transformer (Harvard NLP)

---

## 🎉 Поздравляем!

Вы освоили основы Transformer архитектуры!

Теперь вы понимаете:
- 🧠 Как работает Self-Attention
- 🔮 Почему Transformers революционизировали ML
- 📊 Как применять Transformers к табличным данным
- 🔍 Как интерпретировать модель через attention weights

**Следующий шаг:** TabTransformer для улучшенной работы с категориальными features!
